<a href="https://colab.research.google.com/github/d-youda/Deep_Learning/blob/main/Article_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.9 --upgrade
!pip install keras-cv --upgrade
!pip install git+https://github.com/keras-team/keras-cv.git
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

import pandas as pd
import tensorflow as tf
import keras
import re
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/keras-team/keras-cv.git to /tmp/pip-req-build-fnejfup5
  Running command git clone -q https://github.com/keras-team/keras-cv.git /tmp/pip-req-build-fnejfup5
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 3 not upgraded.
Need to get 430 MB of archives.
After thi

In [ ]:
#한글 데이터 가져오기기
train_text = open('../content/drive/MyDrive/Fashion AI/Article_data.txt','rb').read().decode(encoding='utf-8')
#텍스트 몇 자인지 확인하기
print('Length of text:{} characters'.format(len(train_text)))

#처음 50자 확인하여 데이터 잘 출력되는지확인하기
print(train_text[:50])

Length of text:41409 characters
MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 
글로벌 럭셔리 패션 하우스 


In [ ]:
#훈련 데이터 정제하기기
def clean_str(string):
  string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'/']", " ",string)
  string = re.sub(r","," , ", string)
  string = re.sub(r"!"," ! ",string)
  string = re.sub(r"\(","",string)
  string = re.sub(r"\)","",string)
  string = re.sub(r"\?"," \? " ,string)
  string = re.sub(r"\s{2,}", " " , string)
  string = re.sub(r"\{2,}","\'",string)
  string = re.sub(r"\'","",string)
  string = re.sub(r"\r" , "",string)
  string = re.sub(r"\r " , "",string)
  return string

In [ ]:
train_text = train_text.split("\n")#줄바꿈 기준으로 하나의 string으로 구분
train_test = [clean_str(sentence) for sentence in train_text]
train_text_X = []
for sentence in train_text:
  #띄어쓰기로 단어 구분
  train_text_X.extend(sentence.split(' '))
  train_text_X.append('\n')

train_text_X = [word for word in train_text_X if word != '']

#분류 확인인
print(train_text_X[:20])

['MCM이', '축구', '문화를', '선도하는', '브랜드', '오버더피치와', '만났다.', '\r', '\n', '글로벌', '럭셔리', '패션', '하우스', 'MCM은', '국내', '유명', '풋볼', '크리에이티브', '스튜디오이자', '매거진이면서,']


In [ ]:
#직접 토큰화
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print('{} unique words'.format(len(vocab)))

#vocab list 숫자로 매핑, 반대도 실행하기
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word= np.array(vocab)

text_as_int = np.array([word2idx[c] for c in train_text_X])

#test_as_int 저장된 토큰 데이터 인덱스 확인
print(train_text_X[:20])
print(text_as_int[:20])

5664 unique words
['MCM이', '축구', '문화를', '선도하는', '브랜드', '오버더피치와', '만났다.', '\r', '\n', '글로벌', '럭셔리', '패션', '하우스', 'MCM은', '국내', '유명', '풋볼', '크리에이티브', '스튜디오이자', '매거진이면서,']
[ 399 4799 2180 2792 2522 3618 1956    1    0 1072 1813 5189 5380  397
 1015 3836 5292 4987 3093 2004]


In [ ]:
#25개의 단어 주어졌을 때, 다음 1단어를 예측하는 데이터 만들기
seq_length = 25
example_per_epoch = len(text_as_int)// seq_length
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sentence_dataset = sentence_dataset.batch(seq_length+1,drop_remainder=True)
for item in sentence_dataset.take(1):
  print(idx2word[item.numpy()])
  print(item.numpy())

['MCM이' '축구' '문화를' '선도하는' '브랜드' '오버더피치와' '만났다.' '\r' '\n' '글로벌' '럭셔리' '패션'
 '하우스' 'MCM은' '국내' '유명' '풋볼' '크리에이티브' '스튜디오이자' '매거진이면서,' '어패럴' '브랜드를'
 '전개하는' '오버더피치와의' '협업을' '통해']
[ 399 4799 2180 2792 2522 3618 1956    1    0 1072 1813 5189 5380  397
 1015 3836 5292 4987 3093 2004 3434 2540 4290 3619 5510 5070]


In [ ]:
def split_input_target(chunk):
  return [chunk[:-1],chunk[-1]]

train_dataset = sentence_dataset.map(split_input_target)
for x,y in train_dataset.take(1):
  print(idx2word[x.numpy()])
  print(x.numpy())
  print(idx2word[y.numpy()])
  print(y.numpy())


['MCM이' '축구' '문화를' '선도하는' '브랜드' '오버더피치와' '만났다.' '\r' '\n' '글로벌' '럭셔리' '패션'
 '하우스' 'MCM은' '국내' '유명' '풋볼' '크리에이티브' '스튜디오이자' '매거진이면서,' '어패럴' '브랜드를'
 '전개하는' '오버더피치와의' '협업을']
[ 399 4799 2180 2792 2522 3618 1956    1    0 1072 1813 5189 5380  397
 1015 3836 5292 4987 3093 2004 3434 2540 4290 3619 5510]
통해
5070


In [ ]:
#데이터셋의 shuffle , batch 설정
BATCH_SIZE = 128
steps_per_epoch = example_per_epoch // BATCH_SIZE
BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
total_words = len(vocab)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=seq_length),
    tf.keras.layers.LSTM(units=100,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam' , loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 25, 100)           566400    
                                                                 
 lstm_43 (LSTM)              (None, 25, 100)           80400     
                                                                 
 dropout_22 (Dropout)        (None, 25, 100)           0         
                                                                 
 lstm_44 (LSTM)              (None, 100)               80400     
                                                                 
 dense_22 (Dense)            (None, 5664)              572064    
                                                                 
Total params: 1,299,264
Trainable params: 1,299,264
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#모델 학습시키기기
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch,logs):
  output_idx = []
  test_sentence= []

  if epoch % 5 !=0 and epoch !=49:
    return
  test_sentence = train_text[0]

  next_words = 24
  for _ in range(next_words):
    test_text_X = test_sentence.split(' ')[-seq_length:]
    test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X],maxlen=seq_length, padding='pre',value=word2idx['UNK'])

    classes_X = model.predict(test_text_X)
    output_idx = np.argmax(classes_X,axis=0)
    test_sentence += ''+idx2word[output_idx[0]]
    print()
    print(test_sentence)
    print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)
history = model.fit(train_dataset.repeat(),epochs=50,steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb],verbose=2)

Epoch 1/50
1/1 [==============================] - 1s 1s/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 


1/1 [==============================] - 0s 22ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 



1/1 [==============================] - 0s 18ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 




1/1 [==============================] - 0s 18ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 





1/1 [==============================] - 0s 18ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 






1/1 [==============================] - 0s 23ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 







1/1 [==============================] - 0s 25ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 








1/1 [==============================] - 0s 18ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 









1/1 [==============================] - 0s 18ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 










1/1 [==============================] - 0s 21ms/step

MCM이 축구 문화를 선도하는 브랜드 오버더피치와 만났다. 











1/1 [==============================] - 0s 19ms/step

MCM

In [ ]:
#출시했다는 한 문장 시작으로 칼럼 작성 결과
test_sequence = '삼성물산 패션부문의비이커(BEAKER)는 론칭 10주년을 맞아 가수 백예린과 협업 상품을 출시했다. '

next_words = 100
for _ in range(next_words):
  test_text_X = test_sequence.split(' ')[-seq_length:]
  test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
  test_text_X = pad_sequences([test_text_X],maxlen=seq_length, padding='pre',value=word2idx['UNK'])

   #= model.predict_classes(test_text_X)
  classes_X = model.predict(test_text_X)
  output_idx = np.argmax(classes_X,axis=0)
  test_sentence += ''+idx2word[ output_idx[0]]

print(test_sentence)

1/1 [==============================] - 0s 27ms/step
['삼성물산', '패션부문의비이커(BEAKER)는', '론칭', '10주년을', '맞아', '가수', '백예린과', '협업', '상품을', '출시했다.', '', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n',